In [1]:
using MATLAB


In [2]:
x = [1, 2, 3, 4, 5]
y = [4, 5, 6, 7, 8]
z = [1, 5, 2, 7, 8]
mat"""
plot3($x, $y, $z)
"""





2022-05-05 16:54:41.295 MATLAB_maci64[69685:3429997] CoreText note: Client requested name ".SFNS-Regular", it will get Times-Roman rather than the intended font. All system UI font access should be through proper APIs such as CTFontCreateUIFontForLanguage() or +[NSFont systemFontOfSize:].
2022-05-05 16:54:41.295 MATLAB_maci64[69685:3429997] CoreText note: Set a breakpoint on CTFontLogSystemFontNameRequest to debug.
2022-05-05 16:54:41.301 MATLAB_maci64[69685:3429997] CoreText note: Client requested name ".SFNS-Bold", it will get Times-Roman rather than the intended font. All system UI font access should be through proper APIs such as CTFontCreateUIFontForLanguage() or +[NSFont systemFontOfSize:].
2022-05-05 16:54:41.489 MATLAB_maci64[69685:3429997] CoreText note: Client requested name ".SFNS-Regular", it will get Times-Roman rather than the intended font. All system UI font access should be through proper APIs such as CTFontCreateUIFontForLanguage() or +[NSFont systemFontOfSize:].
2022

In [3]:
using LinearAlgebra
# using PyPlot
using Plots
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface;

In [4]:
h = 0.1 
Nx = 6     # number of state
Nu = 1     # number of controls
Tfinal = 8 # final time-50 knot points or something similar (Get Tfinal from approximation) 
Nt = round(Int,Tfinal/h)+1   # number of time steps-52
thist = Array(range(0,h*(Nt-1), step=h));#0:180,step:3.5
n_nlp = (Nx+Nu)*Nt # number of decision variables-364
m_nlp = Nx*(Nt+1) #318
u_idx = [(Nx+Nu)*(i-1) .+ (Nx+1:Nx+Nu) for i = 1:Nt];
x_idx = [(Nx+Nu)*(i-1) .+ (1:Nx) for i = 1:Nt];
Uref = h*ones(Nt);


In [5]:
function CR3BPdynamics(rv) #Three body dynamics in Sun-Earth System

    μ = 9.537e-4
    
    r₁³= ((rv[1] + μ)^2.0     + rv[2]^2.0 + rv[3]^2.0)^1.5; # distance to m1, LARGER MASS
    r₂³= ((rv[1] - 1 + μ)^2.0 + rv[2]^2.0 + rv[3]^2.0)^1.5; # distance to m2, smaller mass
    # r₁³= ((x + μ)^2     + y^2 + z^2)^1.5; # distance to m1, LARGER MASS
    # r₂³= ((x - 1 + μ)^2 + y^2 + z^2)^1.5; # distance to m2, smaller mass

#     rvdot = zeros(6)
    rvdot = zeros(eltype(rv),6)
    rvdot[1:3] = [rv[4];rv[5];rv[6]]
    rvdot[4] = -((1.0 - μ)*(rv[1] + μ)/r₁³) - (μ*(rv[1] - 1.0 + μ)/r₂³) + 2.0*rv[5] + rv[1];
    rvdot[5] = -((1.0 - μ)*rv[2]      /r₁³) - (μ*rv[2]          /r₂³) - 2.0*rv[4] + rv[2];
    rvdot[6] = -((1.0 - μ)*rv[3]      /r₁³) - (μ*rv[3]          /r₂³);
    return rvdot
    # return [rv[4];rv[5];rv[6];-((1.0 - μ)*(rv[1] + μ)/r₁³) - (μ*(rv[1] - 1.0 + μ)/r₂³) + 2.0*rv[5] + rv[1];-((1.0 - μ)*rv[2]      /r₁³) - (μ*rv[2]          /r₂³) - 2.0*rv[4] + rv[2];-((1.0 - μ)*rv[3]      /r₁³) - (μ*rv[3]          /r₂³)]
end

CR3BPdynamics (generic function with 1 method)

In [6]:
# Linear Approximation
function lin_approx(tau)
    omega = 2.0152105515;
    scaling_factor = 1.495978714e8;
    w_p = 2.086453455;
    w_v = 2.0152105515;
    Ax = 206000/scaling_factor;
    Ay = 665000/scaling_factor;
    Az = 110000/scaling_factor;
    xpoint = 0.932385;
    ypoint = 0;
    zpoint = 0;
    mu = -9.537e-4;
    phi = 0;
    m = 1;
    psi = m*(pi/2) + phi;
    x1 = -Ax*cos(tau + phi) + xpoint;
    y1 = Ay*sin(tau + phi) + ypoint;
    z1 = Az*sin(tau + 1*(pi/2) + phi) + zpoint;
    return [x1, y1, z1]
end 


lin_approx (generic function with 1 method)

In [193]:
# Calc of Reference Trajectory 
Xref =[zeros(Nx) for i=1:Nt];

for k = 1:Nt
    Pos = lin_approx(thist[k]);
    Xref[k][1] = Pos[1];
    Xref[k][2] = Pos[2];
    Xref[k][3] = Pos[3];
    Vels = ForwardDiff.derivative(lin_approx,thist[k]);
    Xref[k][4] = Vels[1];
    Xref[k][5] = Vels[2];
    Xref[k][6] = Vels[3];
end

A1 = ForwardDiff.jacobian(x_ -> CR3BPdynamics(x_), Xref[1]);
A2 = ForwardDiff.jacobian(x_ -> CR3BPdynamics(x_), Xref[2]);

eigvals1 = eigvals(A1)
eigvals2 = eigvals(A2)

eigvecs1 = eigvecs(A1)
eigvecs2 = eigvecs(A2)

# @show size(eigvals1)
# @show size(eigvals2)

# @show size(eigvecs1)
# @show size(eigvecs2)

# @show real(eigvals1[1])
# @show real(eigvals1)
# @show imag(eigvals1)
# @show real(eigvals2)
# @show imag(eigvals2)

eigvalues = [zeros(Nx) for i=1:Nt]
eigs1 = zeros(Nt)
eigs2 = zeros(Nt)
eigs3 = zeros(Nt)
eigs4 = zeros(Nt)
eigs5 = zeros(Nt)
eigs6 = zeros(Nt)

Ieigs1 = zeros(Nt)
Ieigs2 = zeros(Nt)
Ieigs3 = zeros(Nt)
Ieigs4 = zeros(Nt)
Ieigs5 = zeros(Nt)
Ieigs6 = zeros(Nt)

for k = 1:Nt
    A_ = ForwardDiff.jacobian(x_ -> CR3BPdynamics(x_), Xref[k]);
    eigvalues[k] = real(eigvals(A_))
    eigs1[k] = real(eigvals(A_))[1]
    eigs2[k] = real(eigvals(A_))[2]
    eigs3[k] = real(eigvals(A_))[3]
    eigs4[k] = real(eigvals(A_))[4]
    eigs5[k] = real(eigvals(A_))[5]
    eigs6[k] = real(eigvals(A_))[6]

    Ieigs1[k] = imag(eigvals(A_))[1]
    Ieigs2[k] = imag(eigvals(A_))[2]
    Ieigs3[k] = imag(eigvals(A_))[3]
    Ieigs4[k] = imag(eigvals(A_))[4]
    Ieigs5[k] = imag(eigvals(A_))[5]
    Ieigs6[k] = imag(eigvals(A_))[6]
end

mat"""
plot($eigs1, '-x')
hold on
plot($eigs2, '-x')
plot($eigs3, '-x')
plot($eigs4, '-x')
plot($eigs5, '-x')
plot($eigs6, '-x')
hold off
title("Real Part of Eigenvalues")

figure
plot($Ieigs1, '-x')
hold on
plot($Ieigs2, '-x')
plot($Ieigs3, '-x')
plot($Ieigs4, '-x')
plot($Ieigs5, '-x')
plot($Ieigs6, '-x')
hold off
title("Imaginary Part of Eigenvalues")
"""


eigvals1





6-element Vector{ComplexF64}:
 -2.6110295709387605 + 0.0im
                 0.0 - 2.135327058654353im
                 0.0 - 2.0634567533377552im
                 0.0 + 2.0634567533377552im
                 0.0 + 2.135327058654353im
    2.61102957093876 + 0.0im

In [140]:
w_p = 2.086453455;
Ax = 206000
Az = 110000

k = 3.2292680962;
c2 = 4.0610735668;
c3 = 3.0200105081;
c4 = 3.0305378797;
λ = sqrt((c2 + sqrt(9*c2^2 - 8*c2))/2);
d1 = ((3*λ^2)/k)*(k*(6*λ^2 - 1) - 2*λ);
a21 = 3*c3*(k^2 - 2)/(4*(1 + 2*c2)); 
a22 = 3*c3/(4*(1 + 2*c2));
a23 = -((3*c3*λ)/(4*k*d1))*(3*(k^3)*λ - 6*k*(k-λ) + 4); 
a24 = -((3*c3*λ)/(4*k*d1))*(3*k*λ + 2);
b21 = -3*c3*λ*(3*k*λ - 4)/(2*d1);
b22 = -3*c3*λ/d1;
d21 = -c3/(2*λ^2);

# @show k
# @show (w_p^2 + 1 + 2*c2)/(2*w_p)
# @show (2*λ)/(λ^2 + 1 - c2)

# s1 = (2*λ*(λ*(1 + k^2) - 2*k))^(-1)*((3/2)*c3*(2*a21*(k^2 - 2) - a23*(k^2 + 2) - 2*k*b21) - (3/8)*c4*(3*k^4 - 8*k^2 + 8))
# s2 = (2*λ*(λ*(1 + k^2) - 2*k))^(-1)*((3/2)*c3*(2*a22*(k^2 - 2) + a24*(k^2 + 2) + 2*k*b22 + 5*d21) + (3/8)*c4*(12 - k^2))
s1 = -8.246608317e-1
s2 = 1.210985938e-1
# @show s1
# @show s2
ν₁ = 0
ν₂ = s1*Ax^2 + s2*Az^2
ν = 1 + ν₁ + ν₂
ν = 1
T = (2*pi)/(w_p*ν)#/sqrt(3)#*59.01860264189096
@show T


# Az = Array(range(0, 5e5, step = 0.5e4));
# T = zeros(length(Az))
# for i = 1:length(Az)
#     ν₂ = s1*Ax^2 + s2*Az[i]^2
#     ν = 1 + ν₁ + ν₂
#     T[i] = (2*pi)/(w_p*ν)/sqrt(3)
# end
# mat"""
# plot($Az, $T, 'bx')
# """
# # Δ = 0.29221
# # l₁ = -15.96560314
# # l₂ =  1.740900800





T = 3.011418870678611


In [144]:
γ = 0.1001090475489518e-1
μ = 0.3040357143e-5
c2 = (1/(γ^3))*(μ + (-1^2)*(((1-μ)*(γ^2))/((1 - γ)^3)))

ω = sqrt((2 - c2 + sqrt(9*(c2^2) - 8*c2))/2)
# ω = 2.086453455;

T = (2*pi)/ω


# @show (w_p^2 + 1 + 2*c2)/(2*w_p)




T * 365 = 161.5572968895016


161.5572968895016

In [119]:
# Sun Earth System
# m1 = 
# m2 = 
μ = 3.040423398444176e-6
r1 = (1 - (μ/3)^(1/3), 0)
r2 = (1 + (μ/3)^(1/3), 0)
r3 = (1 + (5/12)*μ, 0)
# r4 = ((m1 - μ)/2,  sqrt(3)/2)
# r5 = ((m1 - μ)/2, -sqrt(3)/2)

@show r1
@show r2
@show r3




r1 = (0.989955285350833, 0)
r2 = (1.010044714649167, 0)
r3 = (1.0000012668430827, 0)


(1.0000012668430827, 0)